Распределительная задача. 

Завод может производить $Y$ видов продукции; каждый продукт производится со скоростью $p_i$.
Завод может производить за день только один тип продукции, он может сразу же перейти на производство другого типа продукции, заплатив при этом $R$.
При этом, время всего производства - $S$ дней.

Требуется найти оптимальное с точки зрения максимизации итоговой стоимости производства завода решение, какие продукции изготавливать.
На продукцию есть покупатель, которому требуется определенное количество каждого вида, задаваемое массивом значений $M$.
Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции, задаваемая массивом $C$.

Требуется найти оптимальное с точки зрения максимизации итоговой стоимости производства решение.

Определить, какие типы продукции будут куплены, и какие нет.

Запишем математическую модель.

Введем переменные:

$ x_{i} = \left\{
\begin{array}{ll}
          1, & \mbox { если покупатель заключает контракт на $i$ предмет,}\\
          0, & \mbox { в противном случае. } \\
\end{array} \right. $

Договоримся о входных данных:

$Y$ - количество типов продукции;

$R$ - стоимость перехода с производства одной продукции на производство другой.

$S$ - время производства;

$p_i$ - скорость производства $i$ продукции;

$m_i$ - объем $i$ типа продукции, необходимый покупателю;

$c_i$ - стоимость $i$ продукции.

Математическая модель:

Целевая функция:

$$R + \sum\limits_{i = 0}^{Y - 1} x_i (c_i - R) \to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i : 0 \leqslant i \leqslant Y - 1 $$

$$\sum\limits_{i = 0}^{Y - 1} x_i \frac{m_i}{p_i} \leqslant S $$

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import time
from random import randrange
import pulp as plp
from math import ceil

Using license file /Users/andrlluuk/Desktop/gurobi.lic
Academic license - for non-commercial use only
No parameters matching '_test' found


In [2]:
Y = 12
R = 52
S = 20
M = np.array([26, 46, 66, 47, 31, 47, 34, 31, 49, 37, 38, 63])
P = np.array([26, 25, 20, 25, 20, 16, 22, 23, 20, 16, 29, 25])
C = np.array([509, 512, 635, 433, 511, 678, 408, 665, 462, 620, 563, 659])


In [3]:
problem = plp.LpProblem(name='Distribution problem', sense=plp.LpMaximize)
x  = {i : plp.LpVariable(cat=plp.LpBinary, name="x"+str(i)) for i in range(Y)}

problem.addConstraint(plp.LpConstraint(
                     e=plp.lpSum(M[i]/P[i] * x[i] for i in range(Y)),
                     sense=plp.LpConstraintLE,
                     rhs=S ,
                     name="constraint_{1}"))

problem.setObjective(plp.lpSum((C[i] - R) * x[i] for i in range(Y)) + R)
print(problem)
for i in range(Y):
    print(M[i]/P[i])

Distribution_problem:
MAXIMIZE
457*x0 + 460*x1 + 511*x10 + 607*x11 + 583*x2 + 381*x3 + 459*x4 + 626*x5 + 356*x6 + 613*x7 + 410*x8 + 568*x9 + 52
SUBJECT TO
constraint_{1}: x0 + 1.84 x1 + 1.31034482759 x10 + 2.52 x11 + 3.3 x2 + 1.88 x3
 + 1.55 x4 + 2.9375 x5 + 1.54545454545 x6 + 1.34782608696 x7 + 2.45 x8
 + 2.3125 x9 <= 20

VARIABLES
0 <= x0 <= 1 Integer
0 <= x1 <= 1 Integer
0 <= x10 <= 1 Integer
0 <= x11 <= 1 Integer
0 <= x2 <= 1 Integer
0 <= x3 <= 1 Integer
0 <= x4 <= 1 Integer
0 <= x5 <= 1 Integer
0 <= x6 <= 1 Integer
0 <= x7 <= 1 Integer
0 <= x8 <= 1 Integer
0 <= x9 <= 1 Integer

1.0
1.84
3.3
1.88
1.55
2.9375
1.5454545454545454
1.3478260869565217
2.45
2.3125
1.3103448275862069
2.52


/Users/andrlluuk/opt/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1195: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [4]:
problem.solve(plp.get_solver('PULP_CBC_CMD', msg=False))
for v in problem.variables():
            print(v.name, "=", v.varValue)

x0 = 1.0
x1 = 1.0
x10 = 1.0
x11 = 1.0
x2 = 1.0
x3 = 1.0
x4 = 1.0
x5 = 1.0
x6 = 0.0
x7 = 1.0
x8 = 0.0
x9 = 1.0


In [5]:
def experiment(name_of_solver, solver):
    start = time.time()
    problem.solve(solver)
    answer = plp.value(problem.objective)
    print(f"{name_of_solver} solved {time.time() - start}  answer : {answer}")
    
experiment('Gurobi', plp.get_solver('GUROBI', msg=False))
experiment('CPLEX', plp.get_solver('CPLEX_PY', msg=False))
experiment('PULP', plp.get_solver('PULP_CBC_CMD', msg=False))

Gurobi solved 0.11761999130249023  answer : 5317.0
CPLEX solved 0.12035703659057617  answer : 5317.0
PULP solved 0.0686800479888916  answer : 5317.0


In [6]:
def experiment_per_minute(name_of_solver, solver, TimeLimit):
    var = 0;
    start = time.time()
    currentTime = start
    while(currentTime - start) <= TimeLimit:
        problem.solve(solver)
        var += 1
        currentTime = time.time()
    
    answer = plp.value(problem.objective)
    print(f"{name_of_solver} solved {var} instances "
          f"in {TimeLimit} seconds answer : {answer}")
    
experiment_per_minute('Gurobi', plp.get_solver('GUROBI', msg=False),60)
experiment_per_minute('CPLEX', plp.get_solver('CPLEX_PY', msg=False),60)
experiment_per_minute('PULP', plp.get_solver('PULP_CBC_CMD', msg=False),60)

Gurobi solved 57163 instances in 60 seconds answer : 5317.0
CPLEX solved 6291 instances in 60 seconds answer : 5317.0
PULP solved 2062 instances in 60 seconds answer : 5317.0


QUESTIONS:
1.Как изменится целевая функция, если добавить ограничение: изменить тип выпускаемой продукции в течение дня нельзя!
2.Предположим, что теперь покупатель хочет некоторые типы продукции покупать, а некоторые нет. Как изменится при этом модель?
3.Теперь у покупателя ограниченное количество денег = MONEY. Как изменится модель?
4.Теперь завод может работать только 5 дней в неделю с понедельника по пятницу, два выходных - суббота, воскресенье, но заказ все же нужно сделать в срок S дней.
Как изменится модель?(Начинать работу можно с любого дня недели)
5.Заменим стоимость контракта к каждому типу продукции на стоимость продукции за единицу массы. Как изменится модель?
